<a href="https://colab.research.google.com/github/mahmoudta74/Pytorch_Tutorial/blob/master/deep_lizard_pytorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch.optim as optim

torch.set_printoptions(linewidth=120) #Disply Option for output
torch.set_grad_enabled(True)          # already on by default\


In [ ]:
print(torch.__version__)
print(torchvision.__version__)

1.7.1+cu101
0.8.2+cu101


In [ ]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) #in_channels=1 (number of color channel)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
      
      # (1) input layer
      t = t

      # (2) conv1
      t = self.conv1(t)
      t = F.relu(t)             # or---> t = F.relu(self.conv1(t))
      t = F.max_pool2d(t, kernel_size=2, stride=2)
      
      # (3) conv2
      t = self.conv2(t)
      t = F.relu(t)
      t = F.max_pool2d(t, kernel_size=2, stride=2)

      # (4) hidden linear layer
      t = t.reshape(-1, 12 * 4 * 4) # 12 in the reshaping operation is determined by the number
      t = self.fc1(t)               # of output channels coming from the previous convolutional layer
      t = F.relu(t)                 # 4*4 is actually the height and width of each of the 12 output channels.
                                    # 28*28 reduced to 4*4 by using conv and polling operation.

      # (5) hidden linear layer
      t = self.fc2(t)
      t = F.relu(t)

      # (6) output layer
      t = self.out(t)
      #t = F.softmax(t, dim=1)
      
      return t
        
    def __repr__(self):
      return "Mahmoudnet"

In [ ]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()])
                                              )

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
%load_ext tensorboard

In [ ]:
%matplotlib inline

tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)  # tag = 'images'
tb.add_graph(network, images)
#tb.flush()
tb.close()

In [ ]:
tensorboard  --logdir=runs

In [17]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch

        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)


# Scalar is a number so we add numbers by add_scaler method
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

# 
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram(
        'conv1.weight.grad'
        ,network.conv1.weight.grad
        ,epoch
    )

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

tb.close()

epoch 0 total_correct: 46652 loss: 352.65682473778725
epoch 1 total_correct: 51425 loss: 232.65066657960415
epoch 2 total_correct: 51919 loss: 215.44628629088402
epoch 3 total_correct: 52135 loss: 210.9139289110899
epoch 4 total_correct: 52366 loss: 204.50133468210697
epoch 5 total_correct: 52574 loss: 199.82300221920013
epoch 6 total_correct: 52540 loss: 198.7275465130806
epoch 7 total_correct: 52620 loss: 196.6737863495946
epoch 8 total_correct: 52770 loss: 194.8195599168539
epoch 9 total_correct: 52853 loss: 190.41903398931026


In [ ]:
tensorboard  --logdir=runs

In [18]:
'''
batch_size = 100
lr = 0.01

network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)


comment = f'batch_size= {batch_size} lr={lr}'
tb = SummaryWriter(comment= comment)

tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch

        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item() * batch_size
        total_correct += get_num_correct(preds, labels)


# Scalar is a number so we add numbers by add_scaler method
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

# 
#    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
#    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
#    tb.add_histogram('conv1.weight.grad',network.conv1.weight.grad,epoch)

for name, weight in network.named_parameters():
  tb.add_histogram(name, weight, epoch)
  tb.add_histogram(f'{name}.grad', weight.grad, epoch)


    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

tb.close()
'''

In [ ]:
batch_size_list = [100, 1000, 10000]
lr_list = [.01, .001, .0001, .00001]

In [ ]:
for batch_size in batch_size_list:
    for lr in lr_list:
        network = Network()

        train_loader = torch.utils.data.DataLoader(
            train_set, batch_size=batch_size
        )
        optimizer = optim.Adam(
            network.parameters(), lr=lr
        )

        images, labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)

        comment=f' batch_size={batch_size} lr={lr}'
        tb = SummaryWriter(comment=comment)
        tb.add_image('images', grid)
        tb.add_graph(network, images)

        for epoch in range(5):
            total_loss = 0
            total_correct = 0
            for batch in train_loader:
                images, labels = batch # Get Batch
                preds = network(images) # Pass Batch
                loss = F.cross_entropy(preds, labels) # Calculate Loss
                optimizer.zero_grad() # Zero Gradients
                loss.backward() # Calculate Gradients
                optimizer.step() # Update Weights

                total_loss += loss.item() * batch_size
                total_correct += get_num_correct(preds, labels)

            tb.add_scalar(
                'Loss', total_loss, epoch
            )
            tb.add_scalar(
                'Number Correct', total_correct, epoch
            )
            tb.add_scalar(
                'Accuracy', total_correct / len(train_set), epoch
            )

            for name, param in network.named_parameters():
                tb.add_histogram(name, param, epoch)
                tb.add_histogram(f'{name}.grad', param.grad, epoch)

            print(
                "epoch", epoch
                ,"total_correct:", total_correct
                ,"loss:", total_loss
            )  
        tb.close()

In [ ]:
################################################

## Adding More Hyperparameters Without Nesting

In [21]:
from itertools import product

In [22]:
parameters = dict(
    lr = [.01, .001]
    ,batch_size = [100, 1000]
    ,shuffle = [True, False]
)

In [23]:
param_values = [v for v in parameters.values()]
param_values

[[0.01, 0.001], [100, 1000], [True, False]]

In [24]:
for lr, batch_size, shuffle in product(*param_values): 
    print (lr, batch_size, shuffle)


0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False


In [25]:
for lr, batch_size, shuffle in product(*param_values): 
    comment = f' batch_size={batch_size} lr={lr} shuffle={shuffle}'

    train_loader = torch.utils.data.DataLoader(
        train_set
        ,batch_size=batch_size
        ,shuffle=shuffle 
    )

    optimizer = optim.Adam(
        network.parameters(), lr=lr
    )

    network = Network()

    images, labels = next(iter(train_loader))
    grid = torchvision.utils.make_grid(images)

    tb = SummaryWriter(comment=comment)
    tb.add_image('images', grid)
    tb.add_graph(network, images)

    for epoch in range(5):
        total_loss = 0
        total_correct = 0
        for batch in train_loader:
            images, labels = batch # Get Batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            total_loss += loss.item() * batch_size
            total_correct += get_num_correct(preds, labels)

        tb.add_scalar('Loss', total_loss, epoch)
        tb.add_scalar('Number Correct', total_correct, epoch)
        tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

        for name, param in network.named_parameters():
            tb.add_histogram(name, param, epoch)
            tb.add_histogram(f'{name}.grad', param.grad, epoch)

        print(
                "epoch", epoch
                ,"total_correct:", total_correct
                ,"loss:", total_loss
            )  
    tb.close()

epoch 0 total_correct: 46663 loss: 35113.365480303764
epoch 1 total_correct: 51011 loss: 24234.71935838461
epoch 2 total_correct: 51677 loss: 22297.37410545349
epoch 3 total_correct: 52204 loss: 21076.16904079914
epoch 4 total_correct: 52335 loss: 20573.201489448547
epoch 0 total_correct: 47443 loss: 32895.65112888813
epoch 1 total_correct: 51602 loss: 22701.62342786789
epoch 2 total_correct: 52271 loss: 20664.39869105816
epoch 3 total_correct: 52690 loss: 19828.425781428814
epoch 4 total_correct: 52864 loss: 19330.202239751816
epoch 0 total_correct: 36799 loss: 60367.55186319351
epoch 1 total_correct: 47579 loss: 32497.557133436203
epoch 2 total_correct: 50254 loss: 26842.14437007904
epoch 3 total_correct: 51578 loss: 23296.66566848755
epoch 4 total_correct: 52170 loss: 21433.891594409943
epoch 0 total_correct: 37901 loss: 58198.84669780731
epoch 1 total_correct: 47864 loss: 31861.603021621704
epoch 2 total_correct: 50391 loss: 26039.30401802063
epoch 3 total_correct: 51626 loss: 2273

In [ ]:
tensorboard  --logdir=runs